# Ejemplo DFT

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 23 de marzo de 2025

En este ejemplo se muestra cómo calcular la DFT de una imagen usando OpenCV.

También muestra como editar una imagen en el espacio de las frecuencias modificando los coeficientes y obteniendo la imagen modificada con la DFT inversa.

### Funciones auxiliares
Esta funciones aparecen en los ejemplos de la semana anterior 

In [ ]:
import sys
import os
import numpy
import cv2
from PyQt5.QtWidgets import QApplication, QFileDialog

def agregar_texto(imagen, texto):
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.7
    fontThickness = 2
    tsize, baseline = cv2.getTextSize(texto, fontFace, fontScale, fontThickness)
    pos_text = (int((imagen.shape[1] - tsize[0]) / 2), int(20 + tsize[1]))
    pos_tl = (int(pos_text[0]), int(pos_text[1] - tsize[1]))
    pos_br = (int(pos_tl[0] + tsize[0]), int(pos_tl[1] + tsize[1] + baseline - 2))
    cv2.rectangle(imagen, pos_tl, pos_br, (210,210,210), -1)
    fontColor = (50, 50, 50)
    cv2.putText(imagen, texto, pos_text, fontFace, fontScale, fontColor, fontThickness, cv2.LINE_AA)

def mostrar_imagen(window_name, imagen, texto = ""):
    MAX_WIDTH = 1000
    MAX_HEIGHT = 800
    imagen2 = None
    if imagen.shape[0] > MAX_HEIGHT or imagen.shape[1] > MAX_WIDTH:
        #reducir tamaño
        fh = MAX_HEIGHT / imagen.shape[0]
        fw = MAX_WIDTH / imagen.shape[1]
        escala = min(fh, fw)
        imagen2 = cv2.resize(imagen, (0,0), fx=escala, fy=escala)
    if texto != "":
        if imagen2 is None:
            imagen2 = numpy.copy(imagen)
        agregar_texto(imagen2, texto)
    #mostrar en pantalla
    if imagen2 is None:
        cv2.imshow(window_name, imagen)
    else:
        cv2.imshow(window_name, imagen2)

def ui_select_video():
    app = QApplication(list());
    options = QFileDialog.Options()
    filename, _ = QFileDialog.getOpenFileName(None, "Videos", ".", "Videos (*.mp4 *.mpg *.avi)", options=options)
    if not filename:
        filename = "0" # id de la webcam: 0=primera webcam, 1=segunda webcam
    return filename

def abrir_video(filename):
    if filename is None:
        filename = 0
    elif filename.isdigit():
        filename = int(filename)
    capture = None
    if isinstance(filename, int):
        print("abriendo webcam {}...".format(filename))
        capture = cv2.VideoCapture(filename, cv2.CAP_DSHOW)
    elif (os.path.isfile(filename)):
        print("abriendo video {}...".format(filename))
        capture = cv2.VideoCapture(filename)
    if capture is None or not capture.isOpened():
        raise Exception("no puedo abrir video {}".format(filename))
    return capture;

print("Usando OpenCV {} con Python {}.{}.{}".format(cv2.__version__, sys.version_info.major, sys.version_info.minor,
                                                    sys.version_info.micro))

## Ejemplo DFT de un video o webcam

Selecciona un archivo y muestra la DFT de cada frame (version original y version centrada con color).

Además se muestra la transformada inversa de la DFT, que por defecto es la misma imagen original.

Se puede modificar la imagen en el dominio de las frecuencias con las siguientes teclas:

  * `i`, `k`: sumar o restar una constante a la parte real (cosenos)
  * `o`, `l`: sumar o restar una constante a la parte imaginaria (senos)
  * `p`: intercambiar pesos parte real e imaginaria

Se puede modificar los bordes de la zona de frecuencias a modificar:
  * `a`, `w`, `s`, `d`: mover la esquina top-left de la zona de modificación
  * `g`, `y`, `h`, `j`: mover la esquina bottom-right de la zona de modificación
 
Por ejemplo, al mostrar el video:
  1. Presionar la tecla `i`: aparecerá el resultado de modificar la coordenada [10, 10] de las frecuencias (aparecerá un patrón sobre la imagen)
  2. Presionar la tecla `h` tres veces: se modificarán las coordenada [10:13, 10] 
  3. Presionar la tecla `j` tres veces: se modificarán las coordenada [10:13, 10:13] 


In [ ]:
class Zona:
    def __init__(self, imagen_shape):
        # limites de la imagen
        self.x_max = imagen_shape[1]
        self.y_max = imagen_shape[0]
        # coordenadas rectangulo de modificacion
        # por defecto modifica el pixel 10,10
        self.x_desde = 10
        self.x_hasta = 11 #sin incluir
        self.y_desde = 10
        self.y_hasta = 11 #sin incluir
        # valor a sumar a la parte real o imaginaria
        # por defecto no suma nada
        self.suma_parte_real = 0
        self.suma_parte_imag = 0

    def ajustar_x_desde(self, delta):
        if 0 <= self.x_desde + delta <= self.x_hasta:
            self.x_desde += delta
            return True
        return False

    def ajustar_x_hasta(self, delta):
        if self.x_desde <= self.x_hasta + delta <= self.x_max:
            self.x_hasta += delta
            return True
        return False

    def ajustar_y_desde(self, delta):
        if 0 <= self.y_desde + delta <= self.y_hasta:
            self.y_desde += delta
            return True
        return False

    def ajustar_y_hasta(self, delta):
        if self.y_desde <= self.y_hasta + delta <= self.y_max:
            self.y_hasta += delta
            return True
        return False

    def ajustar_parte_real(self, delta):
        self.suma_parte_real += delta
        return True

    def ajustar_parte_imag(self, delta):
        self.suma_parte_imag += delta
        return True

    def intercambiar_real_imag(self):
        if self.suma_parte_real == self.suma_parte_imag:
            return False
        temp = self.suma_parte_real
        self.suma_parte_real = self.suma_parte_imag
        self.suma_parte_imag = temp
        return True

    def tostring(self):
        coordenadas = "[{}:{}, {}:{}]".format(self.y_desde, self.y_hasta, self.x_desde, self.x_hasta)
        real_con_signo = str(self.suma_parte_real) if self.suma_parte_real <= 0 else "+" + str(self.suma_parte_real)
        imag_con_signo = str(self.suma_parte_imag) if self.suma_parte_imag <= 0 else "+" + str(self.suma_parte_imag)
        ajuste = "({}, {})".format(real_con_signo, imag_con_signo)
        return "pos={} sum={}".format(coordenadas, ajuste)

    def usar_key(self, key):
        # cantidad a ajustar cuando se modifica una posicion
        delta_posicion = 1
        # cantidad a ajustar cuando se modifica un coeficiente
        delta_coeficiente = 10000000
        # probar las teclas
        changed = False
        if key == ord('a'):
            changed = self.ajustar_x_desde(-delta_posicion)
        elif key == ord('d'):
            changed = self.ajustar_x_desde(delta_posicion)
        if key == ord('w'):
            changed = self.ajustar_y_desde(-delta_posicion)
        elif key == ord('s'):
            changed = self.ajustar_y_desde(delta_posicion)
        elif key == ord('g'):
            changed = self.ajustar_x_hasta(-delta_posicion)
        elif key == ord('j'):
            changed = self.ajustar_x_hasta(delta_posicion)
        elif key == ord('y'):
            changed = self.ajustar_y_hasta(-delta_posicion)
        elif key == ord('h'):
            changed = self.ajustar_y_hasta(delta_posicion)
        elif key == ord('k'):
            changed = self.ajustar_parte_real(-delta_coeficiente)
        elif key == ord('i'):
            changed = self.ajustar_parte_real(delta_coeficiente)
        elif key == ord('l'):
            changed = self.ajustar_parte_imag(-delta_coeficiente)
        elif key == ord('o'):
            changed = self.ajustar_parte_imag(delta_coeficiente)
        elif key == ord('p'):
            changed = self.intercambiar_real_imag()
        if changed:
            print(self.tostring())


def modificar_frecuencias(frec_complex, zona):
    if zona.suma_parte_real == 0 and zona.suma_parte_imag == 0:
        return
    frec_complex[zona.y_desde:zona.y_hasta, zona.x_desde:zona.x_hasta, 0] += zona.suma_parte_real
    frec_complex[zona.y_desde:zona.y_hasta, zona.x_desde:zona.x_hasta, 1] += zona.suma_parte_imag

def visualizar_magnitud(frec_complex):
    global global_max_logmagnitud
    # separar complejos en parte real e imaginaria
    frec_real, frec_imag = cv2.split(frec_complex)
    # calcular magnitud
    magnitud = cv2.magnitude(frec_real, frec_imag)
    # calcular log magnitud para visualizar, se usa log(1 + x) para evitar negativos
    logmagnitud = numpy.log1p(magnitud)
    max_encontrado = numpy.max(logmagnitud)
    if max_encontrado > global_max_logmagnitud:
        global_max_logmagnitud = max_encontrado
        print(" nuevo global_max_logmagnitud={}".format(global_max_logmagnitud))
    imageLogMagnitud = cv2.convertScaleAbs(logmagnitud, alpha=255.0 / global_max_logmagnitud)
    return imageLogMagnitud

def imagen_to_complex(frame_gris):
    parte_real = numpy.float32(frame_gris)
    parte_imag = numpy.zeros(frame_gris.shape, numpy.float32)
    frame_complex = cv2.merge([parte_real, parte_imag])
    return frame_complex

def visualizar_centrado_color(imageLogMagnitud):
    # centrar
    w = imageLogMagnitud.shape[1]
    h = imageLogMagnitud.shape[0]
    w2 = int(w / 2)
    h2 = int(h / 2)
    imageCentered = numpy.zeros(imageLogMagnitud.shape, imageLogMagnitud.dtype)
    # copiar regiones
    imageCentered[h2:h, w2:w] = imageLogMagnitud[0:h2, 0:w2]
    imageCentered[h2:h, 0:w2] = imageLogMagnitud[0:h2, w2:w]
    imageCentered[0:h2, w2:w] = imageLogMagnitud[h2:h, 0:w2]
    imageCentered[0:h2, 0:w2] = imageLogMagnitud[h2:h, w2:w]
    # se cambian los grises por escala de colores (0=azul, 255=rojo)
    # Ver https://docs.opencv.org/4.10.0/d3/d50/group__imgproc__colormap.html#ga9a805d8262bcbe273f16be9ea2055a65
    imageCenteredColor = cv2.applyColorMap(imageCentered, cv2.COLORMAP_JET)
    return imageCenteredColor

def procesar_frame(frame, zona, texto):
    # convertir a gris
    frame_gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mostrar_imagen("Original-Gris", frame_gris, texto)
    # convertir la imagen en tipo complejo (2 canales, parte imaginaria=0)
    frame_complex = imagen_to_complex(frame_gris)
    # calcular la DFT de la imagen
    frecuencias_complex = cv2.dft(frame_complex, cv2.DFT_COMPLEX_OUTPUT)
    # modificar las frecuencias de la DFT (edición en el dominio de las frecuencias)
    modificar_frecuencias(frecuencias_complex, zona)
    # visualizar las magnitudes
    imageLogMagnitud = visualizar_magnitud(frecuencias_complex)
    mostrar_imagen("DFTLogMagnitud", imageLogMagnitud)
    imageCenterColor = visualizar_centrado_color(imageLogMagnitud)
    mostrar_imagen("DFTColorCentrado", imageCenterColor)
    # invertir la DFT
    output_frame = cv2.idft(frecuencias_complex, flags=cv2.DFT_SCALE+cv2.DFT_REAL_OUTPUT)
    # mostrar la imagen resultante
    output_frame8 = cv2.convertScaleAbs(output_frame)
    mostrar_imagen("Resultado", output_frame8)

def run_ejemplo(filename):
    zona = None
    capture = abrir_video(filename)
    texto = os.path.basename(filename)
    while capture.grab():
        retval, frame = capture.retrieve()
        if not retval:
            continue
        if zona is None:
            zona = Zona(frame.shape)
        procesar_frame(frame, zona, texto)
        #esperar por una tecla
        key = cv2.waitKey(10)
        # pausa si presiona la barra espaciadora
        if key == ord(' '):
            key = cv2.waitKey(0)
        # salir si presiona q o escape
        if key == ord('q') or key == 27:
            break
        # usar tecla para cambiar la zona
        zona.usar_key(key)
    capture.release()
    cv2.destroyAllWindows()

# se usa un maximo global para escalar las magnitudes al rango 0-255
# se inicia con un global no muy alto y se va a ajustar al maximo encontrado
global_max_logmagnitud = 18;

filename = ui_select_video()
run_ejemplo(filename)

print("FIN")

## Ejemplo DFT de imágenes gaussianas

Una gaussiana es un punto en el centro, el tamaño del punto tiene que ver con al dispersión (sigma) de la gaussiana.

Se prueban distintas gaussianas aumentando el sigma, es decir, imágenes donde se va agrandando el punto central.

Se ve que la DFT de la gaussiana es otra gaussiana.


In [ ]:
def imagen_gaussiana(w, h, sigma):
    pos = numpy.zeros((h,w), numpy.float32)
    for i in range(h):
        for j in range(w):
            x = (h / 2 - i)
            y = (w / 2 - j)
            pos[i, j] = x*x + y*y
    exp = numpy.exp(-numpy.sqrt(pos) / sigma)
    # se escalan linealmente 0 -> 0 y max -> 1
    imag =  exp / numpy.max(exp)
    return cv2.cvtColor(numpy.round(255 * imag).astype(numpy.uint8), cv2.COLOR_GRAY2BGR)


def run_ejemplo2():
    w = 500
    h = 500
    sigma = 0
    zona = Zona((w,h))
    while True:
        if sigma > 400:
            sigma = 0
        sigma += 10
        frame = imagen_gaussiana(w, h, sigma)
        texto = "sigma={}".format(sigma)
        procesar_frame(frame, zona, texto)
        #esperar por una tecla
        key = cv2.waitKey(1)
        # pausa si presiona la barra espaciadora
        if key == ord(' '):
            key = cv2.waitKey(0)
        # salir si presiona q o escape
        if key == ord('q') or key == 27:
            break
        # usar tecla para cambiar la zona
        zona.usar_key(key)
    cv2.destroyAllWindows()

# se usa un maximo global para escalar las magnitudes al rango 0-255
# se inicia con un global no muy alto y se va a ajustar al maximo encontrado
global_max_logmagnitud = 18;

run_ejemplo2()

print("FIN")


## Ejemplo DFT de imágenes con un coseno

Se generan imágenes de patrones coseno con distinta cantidad de ciclos en el eje X e Y.

In [ ]:
def imagen_coseno(w, h, ciclosW, ciclosH):
    pos = numpy.zeros((h, w), numpy.float32)
    for i in range(h):
        for j in range(w):
            a = (j / (w - 1)) * ciclosW
            b = (i / (h - 1)) * ciclosH
            pos[i, j] = a + b
    # coseno entrega valores en [-1, 1], se escala a [0, 1]
    cosenos = (numpy.cos(2 * numpy.pi * pos) + 1) / 2
    # se escalan linealmente -1 -> 0 y 1->255
    return cv2.cvtColor(numpy.round(255 * cosenos).astype(numpy.uint8), cv2.COLOR_GRAY2BGR)

def run_ejemplo3():
    w = 500
    h = 500
    ciclosW = ciclosH = 0
    zona = Zona((w, h))
    while True:
        if ciclosH >= 10:
            ciclosW = ciclosH = 0
            break
        elif ciclosW >= 10:
            ciclosW = 0
            ciclosH += 1
        else:
            ciclosW += 1
        frame = imagen_coseno(w, h, ciclosW, ciclosH)
        texto = "ciclosW={} ciclosH={}".format(ciclosW, ciclosH)
        procesar_frame(frame, zona, texto)
        #esperar por una tecla
        key = cv2.waitKey(10)
        # pausa si presiona la barra espaciadora
        if key == ord(' '):
            key = cv2.waitKey(0)
        # salir si presiona q o escape
        if key == ord('q') or key == 27:
            break
        # usar tecla para cambiar la zona
        zona.usar_key(key)
    cv2.destroyAllWindows()

# se usa un maximo global para escalar las magnitudes al rango 0-255
# se inicia con un global no muy alto y se va a ajustar al maximo encontrado
global_max_logmagnitud = 18;

run_ejemplo3()

print("FIN")
